In [1]:
import os
import fnmatch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from natsort import natsorted

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tflite as lite
from tensorflow.keras.layers import Dense, Flatten,Dropout, BatchNormalization, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import regularizers
from tensorflow_addons.metrics import RSquare
from imgaug import augmenters as img_aug

import warnings
import random
import cv2
import pickle
import requests
import time
import pathlib
import re

warnings.filterwarnings('ignore')

Convert Model:

In [8]:
converter = tf.lite.TFLiteConverter.from_saved_model('model/Nvidia_CNN_angle')
tflite_model = converter.convert()

In [9]:
# Save the model.
with open('model/model_angle.tflite', 'wb') as f:
  f.write(tflite_model)

In [10]:
converter = tf.lite.TFLiteConverter.from_saved_model('model/Nvidia_CNN_speed')
tflite_model = converter.convert()

In [11]:
# Save the model.
with open('model/model_speed.tflite', 'wb') as f:
  f.write(tflite_model)

Test:

In [4]:
def run_tflite_model(tflite_file, test_image):
    interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]["index"], test_image)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_details[0]["index"])
    return predictions

In [5]:
converted_model = "model/model_angle.tflite"
test_dir = '../datasets/PiCar/test/'

On the test set:

In [7]:
# Extract numeric part of file name for sorting
def extract_numeric_part(s):
    return int(re.findall('\d+', s)[0])

# Get list of files in numerical order
files = sorted(os.listdir(test_dir), key=extract_numeric_part)

# Loop through all images in sorted order
for img_name in files:
    img_path = os.path.join(test_dir, img_name)
    img = cv2.imread(img_path)
    resized = cv2.resize(img, (224, 224)).astype('float32')
    resized = resized / 255.
    test_image = np.expand_dims(resized, axis=0)
    prediction = run_tflite_model(converted_model, test_image)
    print("Image:", img_name)
    print("Prediction:", prediction)

Image: 1.png
Prediction: [[0.57352686]]
Image: 2.png
Prediction: [[0.7628894]]
Image: 3.png
Prediction: [[0.23380882]]
Image: 4.png
Prediction: [[0.11036063]]
Image: 5.png
Prediction: [[0.09283561]]
Image: 6.png
Prediction: [[0.77738875]]
Image: 7.png
Prediction: [[0.7020659]]
Image: 8.png
Prediction: [[0.7942834]]
Image: 9.png
Prediction: [[0.5980655]]
Image: 10.png
Prediction: [[0.41017088]]
Image: 11.png
Prediction: [[0.49714264]]
Image: 12.png
Prediction: [[0.74792486]]
Image: 13.png
Prediction: [[0.47363955]]
Image: 14.png
Prediction: [[0.60296106]]
Image: 15.png
Prediction: [[0.16636743]]
Image: 16.png
Prediction: [[0.6695315]]
Image: 17.png
Prediction: [[0.59051603]]
Image: 18.png
Prediction: [[0.7264946]]
Image: 19.png
Prediction: [[0.45831758]]
Image: 20.png
Prediction: [[0.13100587]]
Image: 21.png
Prediction: [[0.9438023]]
Image: 22.png
Prediction: [[0.21061586]]
Image: 23.png
Prediction: [[0.17868622]]
Image: 24.png
Prediction: [[0.7460843]]
Image: 25.png
Prediction: [[0.743

KeyboardInterrupt: 